In [1]:
import pandas as pd
import numpy as np
from dateutil import parser
import re

## Read the Data

In [2]:
data = pd.read_csv('/Users/sejal/Downloads/US_Accidents_March23.csv')

In [3]:
data.head(5)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
ny_data = data.query("State == 'NY'")
ny_data.head(5)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
194250,A-194257,Source2,3,2016-11-30 16:25:41,2016-11-30 17:10:30,41.139324,-74.169876,NaN,NaN,0.01,...,False,False,False,False,False,False,Day,Day,Day,Day
194251,A-194258,Source2,3,2016-11-30 15:40:06,2016-11-30 17:09:40,41.118748,-74.159744,NaN,NaN,2.53,...,False,False,False,False,False,False,Day,Day,Day,Day
194252,A-194259,Source2,3,2016-11-30 15:26:18,2016-11-30 16:56:02,41.014034,-73.717239,NaN,NaN,1.17,...,False,False,False,False,False,False,Day,Day,Day,Day
194253,A-194260,Source2,3,2016-11-30 16:14:11,2016-11-30 16:59:00,41.390369,-73.597969,NaN,NaN,0.01,...,False,False,False,False,False,False,Day,Day,Day,Day
194254,A-194261,Source2,3,2016-11-30 16:17:57,2016-11-30 17:02:44,41.390369,-73.597969,NaN,NaN,0.01,...,False,False,False,False,False,False,Day,Day,Day,Day


### Understanding the data

In [5]:
print(ny_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 347960 entries, 194250 to 7728020
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     347960 non-null  object 
 1   Source                 347960 non-null  object 
 2   Severity               347960 non-null  int64  
 3   Start_Time             347960 non-null  object 
 4   End_Time               347960 non-null  object 
 5   Start_Lat              347960 non-null  float64
 6   Start_Lng              347960 non-null  float64
 7   End_Lat                194902 non-null  float64
 8   End_Lng                194902 non-null  float64
 9   Distance(mi)           347960 non-null  float64
 10  Description            347959 non-null  object 
 11  Street                 347312 non-null  object 
 12  City                   347932 non-null  object 
 13  County                 347960 non-null  object 
 14  State                  347960 non-n

In [6]:
print(ny_data.describe())

            Severity      Start_Lat      Start_Lng        End_Lat   
count  347960.000000  347960.000000  347960.000000  194902.000000  \
mean        2.259550      41.560975     -74.556611      41.536836   
std         0.511014       0.999105       1.574805       0.999496   
min         1.000000      40.503048     -79.761920      40.505200   
25%         2.000000      40.758131     -74.088270      40.750500   
50%         2.000000      40.976681     -73.894638      40.918790   
75%         2.000000      42.743330     -73.760542      42.714843   
max         4.000000      45.001892     -71.861031      44.997320   

             End_Lng   Distance(mi)  Temperature(F)  Wind_Chill(F)   
count  194902.000000  347960.000000   345892.000000  276306.000000  \
mean      -74.526908       0.751562       54.840956      49.544803   
std         1.528085       2.124309       19.336263      23.010782   
min       -79.761981       0.000000      -77.800000     -39.000000   
25%       -74.076729       0

In [7]:
row, cols = ny_data.shape
print(f'Rows: {row} | Columns: {cols}')
print(f'Shape of Dataset {ny_data.shape}')

Rows: 347960 | Columns: 46
Shape of Dataset (347960, 46)


In [8]:
null_values = ny_data.isnull().sum()
null_values

ID                            0
Source                        0
Severity                      0
Start_Time                    0
End_Time                      0
Start_Lat                     0
Start_Lng                     0
End_Lat                  153058
End_Lng                  153058
Distance(mi)                  0
Description                   1
Street                      648
City                         28
County                        0
State                         0
Zipcode                       0
Country                       0
Timezone                      0
Airport_Code                347
Weather_Timestamp          1459
Temperature(F)             2068
Wind_Chill(F)             71654
Humidity(%)                2191
Pressure(in)               2984
Visibility(mi)             3081
Wind_Direction             4477
Wind_Speed(mph)           17879
Precipitation(in)         86511
Weather_Condition          2726
Amenity                       0
Bump                          0
Crossing

## Data Cleaning

In [9]:
ny_data = ny_data.copy()

In [10]:
columns_to_drop =['Country', 'End_Lat', 'End_Lng', 'Source', 'State', 'Timezone', 'Precipitation(in)', 'Wind_Chill(F)']
ny_data.drop(columns=columns_to_drop, inplace=True)

In [11]:
# Fill missing weather-related values with median
weather_cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)']
for col in weather_cols:
    ny_data[col].fillna(ny_data[col].median(), inplace=True)

In [12]:
# Wind speed can be set to the median value for missing entries
ny_data['Wind_Speed(mph)'].fillna(ny_data['Wind_Speed(mph)'].median(), inplace=True)

ny_data['Description'].fillna('No description provided', inplace=True)

## Categorizing the data

In [41]:
# Returning unique list of Weather Condition
weather_list = ny_data['Severity'].unique().tolist()
print("Unique list - ", weather_list)

Unique list -  [3, 2, 4, 1]


In [14]:
weather_mapping = {
    **dict.fromkeys(["Clear", "Fair", "Fair / Windy"], "Clear"),
    **dict.fromkeys(
        [
            "Overcast",
            "Mostly Cloudy",
            "Scattered Clouds",
            "Partly Cloudy",
            "Cloudy",
            "Mostly Cloudy / Windy",
            "Cloudy / Windy",
            "Partly Cloudy / Windy",
        ],
        "Cloudy",
    ),
    **dict.fromkeys(
        [
            "Light Rain",
            "Rain",
            "Heavy Rain",
            "Light Rain / Windy",
            "Rain / Windy",
            "Heavy Rain / Windy",
            "Light Drizzle",
            "Drizzle",
            "Heavy Drizzle",
            "Light Drizzle / Windy",
            "Light Rain with Thunder",
            "Light Rain Showers",
            "Light Rain Shower",
        ],
        "Rain",
    ),
    **dict.fromkeys(
        [
            "Light Snow",
            "Heavy Snow",
            "Snow",
            "Light Snow / Windy",
            "Snow / Windy",
            "Heavy Snow / Windy",
            "Snow and Sleet",
            "Snow and Sleet / Windy",
            "Light Snow and Sleet",
            "Light Snow Showers",
            "Light Snow Shower",
            "Heavy Sleet",
            "Light Sleet",
            "Light Sleet / Windy",
            "Sleet",
            "Sleet / Windy",
            "Wintry Mix",
            "Wintry Mix / Windy",
            "Light Blowing Snow",
            "Blowing Snow",
            "Blowing Snow / Windy",
            "Heavy Snow with Thunder",
            "Snow and Thunder",
            "Light Snow with Thunder",
        ],
        "Snow",
    ),
    **dict.fromkeys(
        [
            "Fog",
            "Mist",
            "Haze",
            "Shallow Fog",
            "Patches of Fog",
            "Patches of Fog / Windy",
            "Haze / Windy",
            "Fog / Windy",
            "Drizzle and Fog",
        ],
        "Fog/Mist/Haze",
    ),
    **dict.fromkeys(
        [
            "Light Thunderstorms and Rain",
            "Thunderstorms and Rain",
            "Heavy Thunderstorms and Rain",
            "Thunderstorm",
            "T-Storm",
            "Heavy T-Storm",
            "Thunder in the Vicinity",
            "Thunder",
            "Thunder / Windy",
            "Heavy T-Storm / Windy",
            "T-Storm / Windy",
            "Light Thunderstorms and Snow",
            "Heavy Thunderstorms and Snow",
            "Thunderstorms and Snow",
        ],
        "Thunderstorms",
    ),
    **dict.fromkeys(
        [
            "Light Freezing Rain",
            "Light Freezing Drizzle",
            "Light Freezing Fog",
            "Ice Pellets",
            "Light Ice Pellets",
            "Freezing Rain",
            "Freezing Drizzle",
            "Smoke",
            "Small Hail",
            "Squalls",
            "Squalls / Windy",
            "N/A Precipitation",
            "Tornado",
        ],
        "Special/Irregular",
    ),
    "nan": "Unknown",
}

In [15]:
ny_data.loc[:, 'Weather_Category'] = ny_data['Weather_Condition'].map(weather_mapping)

In [16]:
print(ny_data[['Weather_Condition', 'Weather_Category']].head())

       Weather_Condition Weather_Category
194250          Overcast           Cloudy
194251          Overcast           Cloudy
194252        Light Rain             Rain
194253              Rain             Rain
194254              Rain             Rain


In [17]:
# Define a custom parsing function that uses dateutil.parser
def robust_parse(dt_str):
    try:
        return parser.parse(dt_str)
    except Exception as e:
        print(f"Failed to parse {dt_str}: {str(e)}")
        return pd.NaT  # Not a Time for unparseable formats


In [18]:
# Apply the robust parsing function to the 'Start_Time' column
ny_data['Start_Time'] = ny_data['Start_Time'].apply(robust_parse)

# Verify the conversion and extract the hour to confirm changes
ny_data['Hour'] = ny_data['Start_Time'].dt.hour

In [32]:
# Extracting different components
ny_data['Date'] = ny_data['Accident_Time'].dt.date  # Date in YYYY-MM-DD
ny_data['Day_Number'] = ny_data['Accident_Time'].dt.day  # Day number of the month
ny_data['Day_of_Week'] = ny_data['Accident_Time'].dt.day_name()  # Day of the week
ny_data['Week_of_Year'] = ny_data['Accident_Time'].dt.isocalendar().week  # Week number of the year
ny_data['Year'] = ny_data['Accident_Time'].dt.year  # Year
ny_data['Month'] = ny_data['Accident_Time'].dt.month_name()  # Month name
ny_data['Month_Number'] = ny_data['Accident_Time'].dt.month  # Month number

In [19]:
# Define bins and labels for the time of day
bins = [-1, 5, 9, 15, 19, 24]  # Bins include the endpoints for hour intervals
labels = ['Late Night to Early Morning', 'Morning Commute', 'Midday', 'Evening Commute', 'Late Evening']

# Categorize time of day using cut
ny_data['Time_of_Day'] = pd.cut(ny_data['Hour'], bins=bins, labels=labels, right=True)

In [20]:
# Display the first few rows of the DataFrame to see the new column
print(ny_data[['Start_Time', 'Time_of_Day']].head())

                Start_Time      Time_of_Day
194250 2016-11-30 16:25:41  Evening Commute
194251 2016-11-30 15:40:06           Midday
194252 2016-11-30 15:26:18           Midday
194253 2016-11-30 16:14:11  Evening Commute
194254 2016-11-30 16:17:57  Evening Commute


In [21]:
# Function to extract road types from descriptions
def extract_road_type(description):
    road_types = {
        'Interstate Highway': r'\b(I-\d+|Interstate \d+)\b',
        'US Highway': r'\b(US-\d+|U.S. Route \d+)\b',
        'US Highway': r'\b(US-\d+|U.S. Route \d+|US-\d+[ Ww])\b',
        'State Highway': r'\b(State|State Route \d+|SR-\d+|State Hwy \d+|NY-\d+|RT-\d+|RT|Hwy|Pky|Parkway|Pkwy|Expy|Expressway|Cswy|Causeway|Turnpike|Tpke|Freeway|Fwy|Bypass|Thruway|Thwy|Exit \d+|at \d+\w{0,2} St|Exit)\b',
    }

    for key, val in road_types.items():
        if re.search(val, description):
            return key
    return 'Local Road'

In [22]:
# Apply the function to the Description column
ny_data['Road_Type'] = ny_data['Description'].apply(extract_road_type)

In [26]:
new_column_names = {
    'Start_Lat': 'Latitude',
    'Start_Lng': 'Longitude',
    'ID': 'Accident_ID',
    'Start_Time': 'Accident_Time'
    # Add more mappings as needed
}

ny_data.columns = [new_column_names.get(col, col) for col in ny_data.columns]

In [35]:
ny_data.head(10)

,Accident_ID,Severity,Accident_Time,End_Time,Latitude,Longitude,Distance(mi),Description,Street,City,...,Hour,Time_of_Day,Road_Type,Date,Day_Number,Day_of_Week,Week_of_Year,Year,Month,Month_Number
194250,A-194257,3,2016-11-30 16:25:41,2016-11-30 17:10:30,41.139324,-74.169876,0.01,Right lane blocked due to accident on I-87 Sou...,Route 17,Sloatsburg,...,16,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
194251,A-194258,3,2016-11-30 15:40:06,2016-11-30 17:09:40,41.118748,-74.159744,2.53,Accident on I-87 Southbound between Exit 14B C...,I-287 S,Suffern,...,15,Midday,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
194252,A-194259,3,2016-11-30 15:26:18,2016-11-30 16:56:02,41.014034,-73.717239,1.17,Accident on I-287 Cross Westchester Expy Eastb...,Hutchinson River Pkwy S,White Plains,...,15,Midday,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
194253,A-194260,3,2016-11-30 16:14:11,2016-11-30 16:59:00,41.390369,-73.597969,0.01,Right hand shoulder blocked due to accident on...,I-84 E,Brewster,...,16,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
194254,A-194261,3,2016-11-30 16:17:57,2016-11-30 17:02:44,41.390369,-73.597969,0.01,Right hand shoulder blocked due to accident on...,I-84 E,Brewster,...,16,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
194255,A-194262,2,2016-11-30 15:30:42,2016-11-30 17:00:28,40.696957,-73.337006,0.01,Accident pedestrian involved on NY-109 Little ...,Little East Neck Rd,Babylon,...,15,Midday,State Highway,2016-11-30,30,Wednesday,48,2016,November,11
194267,A-194274,2,2016-11-30 17:12:40,2016-11-30 17:57:22,40.724514,-73.251434,0.00,Accident pedestrian involved on County Hwy-50 ...,4th Ave,Bay Shore,...,17,Evening Commute,State Highway,2016-11-30,30,Wednesday,48,2016,November,11
194271,A-194278,3,2016-11-30 17:25:30,2016-11-30 18:10:18,42.728401,-73.796188,0.01,Right hand shoulder blocked due to accident on...,I-87 S,Albany,...,17,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
194272,A-194279,3,2016-11-30 17:21:50,2016-11-30 18:06:29,42.643890,-73.750336,0.01,Shoulder blocked on exit ramp due to accident ...,Dunn Memorial Brg,Albany,...,17,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
194273,A-194280,3,2016-11-30 17:25:55,2016-11-30 18:10:44,40.987408,-73.670479,0.01,Accident on I-95 Southbound at Exits 21 22 I-2...,Cross Westchester Expy E,Rye,...,17,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11


In [25]:
rows_with_id = data[data['Accident ID'] == 'A-1228612']
display(rows_with_id)

,Accident ID,Source,Severity,Accident_Time,End_Time,Latitude,Longitude,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
1218831,A-1228612,Source2,2,2020-12-15 07:15:21,2020-12-15 08:04:13,44.38744,-89.799812,NaN,NaN,0.0,...,False,False,True,False,False,False,Night,Day,Day,Day


In [34]:
sorted_column_names = sorted(ny_data.columns)
sorted_column_names

['Accident_ID',
 'Accident_Time',
 'Airport_Code',
 'Amenity',
 'Astronomical_Twilight',
 'Bump',
 'City',
 'Civil_Twilight',
 'County',
 'Crossing',
 'Date',
 'Day_Number',
 'Day_of_Week',
 'Description',
 'Distance(mi)',
 'End_Time',
 'Give_Way',
 'Hour',
 'Humidity(%)',
 'Junction',
 'Latitude',
 'Longitude',
 'Month',
 'Month_Number',
 'Nautical_Twilight',
 'No_Exit',
 'Pressure(in)',
 'Railway',
 'Road_Type',
 'Roundabout',
 'Severity',
 'Station',
 'Stop',
 'Street',
 'Sunrise_Sunset',
 'Temperature(F)',
 'Time_of_Day',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop',
 'Visibility(mi)',
 'Weather_Category',
 'Weather_Condition',
 'Weather_Timestamp',
 'Week_of_Year',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Year',
 'Zipcode']

In [36]:
ny_data.to_csv('NY Accidents.csv', index=False)

In [40]:
total_accidents = ny_data['Accident_ID'].nunique()
print(f"Total number of accidents: {total_accidents}")

Total number of accidents: 347960


In [38]:
ny_data['Cumulative_Accidents'] = ny_data['Accident_Count'].cumsum()

In [39]:
print(ny_data[['Accident_ID', 'Accident_Count', 'Cumulative_Accidents']].head())

       Accident_ID  Accident_Count  Cumulative_Accidents
194250    A-194257               1                     1
194251    A-194258               1                     2
194252    A-194259               1                     3
194253    A-194260               1                     4
194254    A-194261               1                     5


In [42]:
data = pd.read_csv('/Users/sejal/Documents/SpringSem/Intro to ML/NY Accidents.csv')

In [43]:
data.head()

,Accident_ID,Severity,Accident_Time,End_Time,Latitude,Longitude,Distance(mi),Description,Street,City,...,Hour,Time_of_Day,Road_Type,Date,Day_Number,Day_of_Week,Week_of_Year,Year,Month,Month_Number
0,A-194257,3,2016-11-30 16:25:41,2016-11-30 17:10:30,41.139324,-74.169876,0.01,Right lane blocked due to accident on I-87 Sou...,Route 17,Sloatsburg,...,16,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
1,A-194258,3,2016-11-30 15:40:06,2016-11-30 17:09:40,41.118748,-74.159744,2.53,Accident on I-87 Southbound between Exit 14B C...,I-287 S,Suffern,...,15,Midday,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
2,A-194259,3,2016-11-30 15:26:18,2016-11-30 16:56:02,41.014034,-73.717239,1.17,Accident on I-287 Cross Westchester Expy Eastb...,Hutchinson River Pkwy S,White Plains,...,15,Midday,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
3,A-194260,3,2016-11-30 16:14:11,2016-11-30 16:59:00,41.390369,-73.597969,0.01,Right hand shoulder blocked due to accident on...,I-84 E,Brewster,...,16,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11
4,A-194261,3,2016-11-30 16:17:57,2016-11-30 17:02:44,41.390369,-73.597969,0.01,Right hand shoulder blocked due to accident on...,I-84 E,Brewster,...,16,Evening Commute,Interstate Highway,2016-11-30,30,Wednesday,48,2016,November,11


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347960 entries, 0 to 347959
Data columns (total 49 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Accident_ID            347960 non-null  object 
 1   Severity               347960 non-null  int64  
 2   Accident_Time          347960 non-null  object 
 3   End_Time               347960 non-null  object 
 4   Latitude               347960 non-null  float64
 5   Longitude              347960 non-null  float64
 6   Distance(mi)           347960 non-null  float64
 7   Description            347960 non-null  object 
 8   Street                 347312 non-null  object 
 9   City                   347932 non-null  object 
 10  County                 347960 non-null  object 
 11  Zipcode                347960 non-null  object 
 12  Airport_Code           347613 non-null  object 
 13  Weather_Timestamp      346501 non-null  object 
 14  Temperature(F)         347960 non-nu